In [1]:
import sys
sys.path.append('..')

import lcatools
import os

In [3]:
lcia_file = '/data/LCI/LCIA implementation v3.1 2014_08_13.xlsx'
#spold_archive = '/data/LCI/Ecoinvent/3.2/cutoff/datasets'

catalog_dir = os.path.join(os.path.expanduser('~'), 'GitHub', 'lca-tools-datafiles', 'catalogs')
spold_catfile = 'ei3.2_undefined_spold.json.gz'

## benchmark xlrd operation

%time X = xlrd.open_workbook(lcia_file)

In [2]:
import xlrd
import re

In [3]:
from collections import defaultdict

In [10]:
s = X.sheet_by_index(1)

In [11]:
s.name

'impact methods'

In [12]:
b = X.sheet_by_name('impact methods')

In [62]:
headings

['method',
 'category',
 'indicator',
 'name',
 'compartment',
 'subcompartment',
 'CF 3.01',
 'CF 3.1',
 'Change?',
 'note',
 'Known issue']

In [54]:
cfs = []
for row in g:
    d = dict()
    for i,h in enumerate(headings):
        d[h] = row[i]
    cfs.append(d)

In [55]:
len(cfs)

170915

In [56]:
cfs[0]

{'CF 3.01': number:1.6,
 'CF 3.1': number:1.6,
 'Change?': text:'FALSE',
 'Known issue': empty:'',
 'category': text:'acidification potential',
 'compartment': text:'air',
 'indicator': text:'average European',
 'method': text:'CML 2001',
 'name': text:'Ammonia',
 'note': text:'',
 'subcompartment': text:'low population density, long-term'}

## generate ei_lcia

In [4]:
from lcatools.providers.ecoinvent_lcia import EcoinventLcia
from lcatools.db_catalog import from_json
from lcatools.providers.ecospold2 import EcospoldV2Archive

In [5]:
lcia_file

'/data/LCI/LCIA implementation v3.1 2014_08_13.xlsx'

es_upstream = EcospoldV2Archive('/data/LCI/Ecoinvent/3.2/cutoff/datasets', quiet=True)

In [6]:
es_upstream = from_json(os.path.join(catalog_dir, spold_catfile))

19 new quantity entities added (19 total)
4680 new flow entities added (4680 total)
11552 new process entities added (11552 total)


In [7]:
[c for c in es_upstream.flows()[0].characterizations()]

In [8]:
es_upstream.processes()[135].serialize()

{'Classifications': ['ISIC rev.4 ecoinvent: 3821:Treatment and disposal of non-hazardous waste'],
 'Comment': 'This dataset represents the activity of waste disposal of residue from mechanical treatment, laser printer in a municipal solid waste incinerator (MSWI)  //  Recommended use of this dataset: Only for incineration of shredder residues generated in the recycling (mechanical treatment) of laser printers  //  Inventoried waste contains 1.92% rubber; 47.1% Plastics from electronic consumer goods; 47.1% Plastics from electronic industrial goods; 1.42% MSWI iron scrap; 0.151% Alu in MSW; 2.25% wiring copper; .  //  Waste composition (wet, in ppm): upper heating value 34.74 MJ/kg; lower heating value 33.3 MJ/kg; H2O 8644.9; O 50262; H 71805; C 784170; S 411.86; N 12537; P 482.64; B n.a.; Cl 6388.1; Br 5184.3; F 235.65; I n.a.; Ag n.a.; As 7.0695; Ba 166.84; Cd 142.06; Co 27.785; Cr 66.493; Cu 25690; Hg 0.80123; Mn 40.518; Mo 2.8278; Ni 487.12; Pb 144.39; Sb 4523; Se 1.8471; Sn 200.84;

In [9]:
[k['referenceUnit'].unitstring() for k in es_upstream.quantities()]

['person*km',
 'm*year',
 'unit',
 'm3',
 'ha',
 'kWh',
 'km*year',
 'm2*year',
 'm3*year',
 'kBq',
 'm',
 'hour',
 'km',
 'l',
 'metric ton*km',
 'kg',
 'm2',
 'MJ',
 'kg*day']

In [10]:
EL = EcoinventLcia(lcia_file, upstream=es_upstream, mass_quantity=es_upstream.quantity_with_unit('kg'), 
                   ns_uuid='46802ca5-8b25-398c-af10-2376adaa4623')

In [11]:
%time EL._load_xl_rows()

CPU times: user 21.1 s, sys: 40 ms, total: 21.2 s
Wall time: 21.2 s


In [12]:
len(EL._xl_rows)

170915

In [13]:
for row in EL._xl_rows:
    EL._create_flow(row)

In [14]:
import itertools

def group_by(result, group_key):
    keys = []
    groups = []
    for i, j in itertools.groupby(sorted(result, key=group_key),
                                  group_key):
        keys.append(i)
        groups.append(list(j))

    return keys, groups

In [15]:
org, flws = group_by(EL.flows(), lambda x: x['origin'])

In [16]:
org

['/data/LCI/Ecoinvent/3.2/undefined',
 '/data/LCI/LCIA implementation v3.1 2014_08_13.xlsx']

In [17]:
flws[0][1].__dict__

{'_characterizations': {<lcatools.characterizations.Characterization at 0x7f751ee585c0>},
 '_d': {'CasNumber': None,
  'Comment': 'EcoSpold02 Flow',
  'Compartment': ['water', 'surface water'],
  'Name': 'Dipropylamine',
  'externalId': 'f87e0478-9ba4-4d6b-a1ca-5667d07f3465',
  'origin': '/data/LCI/Ecoinvent/3.2/undefined'},
 '_external_ref': 'f87e0478-9ba4-4d6b-a1ca-5667d07f3465',
 '_ref_quantity_factor': 1.0,
 '_uuid': UUID('f87e0478-9ba4-4d6b-a1ca-5667d07f3465'),
 'entity_type': 'flow',
 'reference_entity': <lcatools.entities.LcQuantity at 0x7f751dd38dd8>}

In [12]:
EL._xl_rows[15]

{'CF 3.01': 1.88,
 'CF 3.1': 1.88,
 'Known issue': '',
 'category': 'acidification potential',
 'compartment': 'air',
 'indicator': 'generic',
 'method': 'CML 2001',
 'name': 'Ammonia',
 'note': '',
 'subcompartment': 'low population density, long-term'}

In [18]:
EL[EL._upstream_hash[EL._flow_key(EL._xl_rows[15])].get_uuid()]

In [19]:
EL['2b50f643-216a-412b-a0e5-5946867aa2ed'].__dict__

{'_characterizations': {<lcatools.characterizations.Characterization at 0x7f751f126710>},
 '_d': {'CasNumber': None,
  'Comment': 'EcoSpold02 Flow',
  'Compartment': ['air', 'low population density, long-term'],
  'Name': 'Ammonia',
  'externalId': '2b50f643-216a-412b-a0e5-5946867aa2ed',
  'origin': '/data/LCI/Ecoinvent/3.2/undefined'},
 '_external_ref': '2b50f643-216a-412b-a0e5-5946867aa2ed',
 '_ref_quantity_factor': 1.0,
 '_uuid': UUID('2b50f643-216a-412b-a0e5-5946867aa2ed'),
 'entity_type': 'flow',
 'reference_entity': <lcatools.entities.LcQuantity at 0x7f751dd38dd8>}

In [20]:
qs = set()
for f in EL.flows():
    qs.add(f['referenceQuantity'].get_uuid())

In [21]:
['%s %s' % (k, EL[k]) for k in qs]

['1017f68a-f818-4f9d-a34d-a31e7386f628 LC quantity: EcoSpold Quantity m2',
 '980b811e-3905-4797-82a5-173f5568bc7e LC quantity: EcoSpold Quantity MJ',
 'eb955b7c-7bed-401f-9c76-5db716ca3640 LC quantity: EcoSpold Quantity m2*year',
 '4923348e-591b-4772-b224-d19df86f04c9 LC quantity: EcoSpold Quantity kBq',
 '487df68b-4994-4027-8fdc-a4dc298257b7 LC quantity: EcoSpold Quantity kg',
 'de5b3c87-0e35-4fb0-9765-4f3ba34c99e5 LC quantity: EcoSpold Quantity m3']

In [22]:
EL.check_counter()

0 new process entities added (0 total)
3255 new flow entities added (3255 total)
6 new quantity entities added (6 total)


In [23]:
f = EL.search('30f6b7f3-65b0-3705-b096-a2ef311315c2')[0]

In [24]:
str(f)

'Ammonia [air, lower stratosphere + upper troposphere]'

In [25]:
es_upstream.search(entity_type='flow', Name='Ammonia', Compartment='tropo')

[]

In [26]:
EL.search(entity_type='flow', Name='Ammonia', Compartment='tropo')

In [27]:
EL.flows()[0].__dict__

{'_characterizations': {<lcatools.characterizations.Characterization at 0x7f751f0b81d0>},
 '_d': {'CasNumber': None,
  'Comment': 'EcoSpold02 Flow',
  'Compartment': ['air', 'non-urban air or from high stacks'],
  'Name': 'Krypton-88',
  'externalId': '56ce80e5-9a77-4a80-ad65-6208db560be5',
  'origin': '/data/LCI/Ecoinvent/3.2/undefined'},
 '_external_ref': '56ce80e5-9a77-4a80-ad65-6208db560be5',
 '_ref_quantity_factor': 1.0,
 '_uuid': UUID('56ce80e5-9a77-4a80-ad65-6208db560be5'),
 'entity_type': 'flow',
 'reference_entity': <lcatools.entities.LcQuantity at 0x7f751dd38eb8>}

In [28]:
EL.flows()[0].get_signature()

{'CasNumber': None,
 'Comment': 'EcoSpold02 Flow',
 'Compartment': ['air', 'non-urban air or from high stacks'],
 'EntityType': 'flow',
 'Name': 'Krypton-88',
 'referenceQuantity': <lcatools.entities.LcQuantity at 0x7f751dd38eb8>}

In [29]:
%time EL.load_all()

0 new process entities added (0 total)
0 new flow entities added (3255 total)
701 new quantity entities added (707 total)
CPU times: user 1min 13s, sys: 200 ms, total: 1min 13s
Wall time: 1min 13s


In [30]:
[ '%10.3e [] %s' % (x.value, x.quantity) 
 for x in EL['9d07940b-de52-3114-a800-9c3b26530734']._characterizations 
 if x.value is not None]

[' 3.000e-01 [] LC quantity: EDIP, environmental impact, photochemical ozone formation, high NOx POCP',
 ' 1.400e+04 [] LC quantity: ecological scarcity 2013, total, total',
 ' 2.000e+00 [] LC quantity: EDIP, environmental impact, global warming, GWP 20a',
 ' 1.800e+04 [] LC quantity: ecological scarcity 2006, total, total',
 ' 3.325e-02 [] LC quantity: eco-indicator 99, (H,A), total, total',
 ' 2.000e-01 [] LC quantity: EDIP, environmental impact, photochemical ozone formation, low NOx POCP',
 ' 2.477e-02 [] LC quantity: eco-indicator 99, (E,E), human health, respiratory effects',
 ' 2.477e-02 [] LC quantity: eco-indicator 99, (E,E), total, total',
 ' 3.325e-02 [] LC quantity: eco-indicator 99, (H,A), human health, total',
 ' 3.200e+04 [] LC quantity: ecological scarcity 1997, total, emission into air',
 ' 1.400e+04 [] LC quantity: ecological scarcity 2013, total, Main air pollutants and PM',
 ' 3.894e-02 [] LC quantity: EDIP2003, photochemical ozone formation, impacts on human health

In [31]:
EL['9d07940b-de52-3114-a800-9c3b26530734']['origin']

'/data/LCI/LCIA implementation v3.1 2014_08_13.xlsx'

In [32]:
EL.write_to_file(os.path.join(catalog_dir, 'ei_lcia.json.gz'), gzip=True, characterizations=True, values=True)

In [53]:
[str(c.quantity['Name']) for c in EL.search('56ce8')[0]._characterizations]

['EcoSpold Quantity kBq',
 'selected LCI results, additional, air, radioactive, noble gas']

In [42]:
EL.search('56ce8')[0]['origin']

'/data/LCI/Ecoinvent/3.2/undefined'

In [34]:
EL._mass.get_uuid()

'487df68b-4994-4027-8fdc-a4dc298257b7'

In [54]:
list(EL.search('56ce8')[0]._characterizations)[0].quantity is EL.search('56ce8')[0].reference_entity

True

In [56]:
EL.flows()[100].get_signature()

{'CasNumber': None,
 'Comment': 'EcoSpold02 Flow',
 'Compartment': ['soil', 'agricultural'],
 'EntityType': 'flow',
 'Name': 'Spiroxamine',
 'referenceQuantity': <lcatools.entities.LcQuantity at 0x7f751dd38dd8>}

In [29]:
"""
entity generator-- I need a flow
"""

def en_gen(archive):
    for k in archive._entities.keys():
        v = archive._get_entity(k)
        if v.entity_type == 'flow':
            yield v
u = en_gen(es_upstream)

In [30]:
e = next(u)

In [31]:
next(u)['Compartment']

['natural resource', 'land']

In [32]:
e = next(u)
[e['Name']] + e['Compartment']

['Strontium', 'soil', 'agricultural']

In [33]:
e

In [37]:
unique_names = set([f['Name'] for f in EL.flows()])

In [38]:
len(unique_names)

984

In [39]:
list(unique_names)[:10]

['Metamorphous rock, graphite containing, in ground',
 'Barium-140',
 'Pirimicarb',
 'Zinc-65',
 'Sylvite, 25 % in sylvinite, in ground',
 'Phosphoric acid',
 'Methyl lactate',
 'Polychlorinated biphenyls',
 'Copper, 2.19% in sulfide, Cu 1.83% and Mo 8.2E-3% in crude ore, in ground',
 'Iodine']

In [43]:
[f for f in EL.flows() if f.reference_entity is None]

[]

In [44]:
from lcatools.tools import archive_from_json

In [47]:
g = archive_from_json(os.path.join(catalog_dir, 'ei_lcia.json.gz'))

Loading JSON data from /home/b/GitHub/lca-tools-datafiles/catalogs/ei_lcia.json.gz:
**Upstream reference encountered: /data/LCI/Ecoinvent/3.2/cutoff/datasets

708 new quantity entities added (708 total)
3255 new flow entities added (3255 total)
0 new process entities added (0 total)


In [50]:
[f for f in EL.flows() if f.reference_entity is None]

[]